# Internet2 CloudConnect with AWS Direct Connect using Jupyter Notebooks

<div class="alert alert-info">

**Note:**    AWS Direct Connect functionality requires a more hands-on approach and some manual steps. Assistance from the Chameleon team is likely required, especially if you do not have access to Internet2's CloudConnect portal. Please reach out if you're interested in working on a setup like this.

</div>

## Background

Network configuration is a critical element of any experiment spanning research and commercial clouds. The most common option is to assign domain specific public Internet addresses to all nodes and send traffic over the traditional Internet.  This approach is adequate for many experiment but suffers from security and performance limitations. Another option is to use a virtual private network (VPN) as a tunnel between distributed sites; the VPN protects the architecture from common security attacks and allows remote cloud resources to be assigned local IP addresses and managed as if they were on-site. Both of these options have the advantage of easy implementation, but are limited by the performance of the public Internet, an important consideration for many experiments. 

Many computer science experiments require increased control of the wide area network, include everything from garunteed quality of service to low-level network programability using software defined networking.  Chameleon provides these capabilities though direct low-level network connection between the research and public clouds, however creating them is challenging. While most public clouds provide low-level networking services (e.g. [AWS Direct Connect](https://aws.amazon.com/directconnect/), [Azure ExpressRoute](https://azure.microsoft.com/en-us/services/expressroute/), or [Google Dedicated Interconnect](https://cloud.google.com/network-connectivity/docs/interconnect/concepts/dedicated-overview)), using them is typically expensive; on the research cloud side, they can involve complicated campus network configuration arrangements that often limit access to this type of experimental configuration to a few a few select scientists or campus IT staff themselves. 

Since 2016, the Chameleon testbed has provided direct connect using Internet2’s [Advanced Layer 2 service (AL2S)](https://www.internet2.edu/products-services/advanced-networking/layer-2-services/) via [ExoGENI](http://www.exogeni.net/). More recently, Internet2 has deployed its [CloudConnect](https://www.internet2.edu/products-services/advanced-networking/networking-for-cloud/) service that enables members to connect end points, such as those used for Chameleon direct AL2S connections, to AWS Direct Connect, Azure ExpressRoute, and Google Dedicated Interconnect sites. Thus, to create an experimental topology between Chameleon and commercial cloud the first step is to create a direct connection between Chameleon and a public cloud accessible using Internet2’s CloudConnect. Additionally, since public cloud direct connections configure routing between the cloud and external facility using BGP, a user also needs to deploy a BGP router on their resources. 

This Jupyter notebook walks through the deployment of an experiment spanning Chameleon and AWS using CloudConnect.  It deploys the network, compute servers, and a fully configured BGP router. Further, the BGP router can, optionally, be deployed on a dedicated OpenFlow networking switch or as software Quagga router existing on a standard x86 compute host.  The full networking configuration is depicted in the figure below.

<br>
<center>
    <img src="./CloudConnect-Figs/ChameleonCloudConnectSW.png"><br>
    <em>Experiment spanning Chameleon and AWS using Internet2 CloudConnect</em>
</center>
<br>

The configuration of the experiment can be seen in the figure.  A BGP router was deployed on a Chameleon host connected
to two dedicated 10 Gbps tenant networks. One was an externally connected network that was stitched to an Internet2 CloudConnect BGP router. The other was an internal network connected to other compute nodes on Chameleon. On the AWS side, the CloudConnect BGP router was connected to a Virtual Private Gateway (VPG). The router in a Virtual Private Cloud (VPC) was configured with default routes to a private
Internet Gateway and custom routes through the dedicated Internet2 circuit to the isolated tenant network on Chameleon. The three BGP  routers cooperate to advertise routes between user-controlled subnets hosted on Chameleon and AWS. The infrastructure’s configuration is described in the Chameleon tutorial on using Internet2 CloudConnect.

## Tutorial

There are three components of the deployed infrastructure: Chameleon, AWS, and CloudConnect. Each step is described in the following sections. The configuration parameteres using in the example are shown in the following figure.

<br>
<center>
    <img src="./CloudConnect-Figs/AWS-Config.png"><br>
    <em>Configuration Parameters</em>
</center>
<br>

#### Set the configuration variables

In [35]:
# Project name and targeted region
export OS_PROJECT_NAME='CH-816532'
export OS_REGION_NAME='CHI@UC'

# Any key you have uploaded to Chameleon
export KEY_NAME='my_chameleon_key'

# Location of the matching private key in your Jupyter container
export PRIVATE_KEY_LOCATION="/home/${USER}/work/my_chameleon_key"

# Set a preface to be used to identify named experiment objects in Chameleon
# You username is a good PREFACE.
export PREFACE=${USER}

#Desired name of the direct connect circuit
DIRECT_CONNECT_NAME="Chameleon_Direct_Connect"

# AWS VPC Network
# Name of EXISTING VPC in your AWS account
export AWS_VPC_NAME="Chameleon_Direct_Connect_VPC"
export AWS_INTERNAL_SUBNET="192.168.1.0/24"
export AWS_VLAN=10

# Chameleon-to-AWS Network (i.e. "External Network")
export EXTERNAL_SUBNET="192.168.3.0/24" 
export EXTERNAL_NET_INTERNET_GATEWAY_IP="192.168.3.254"

# Chameleon Internal Network
export CHAMELEON_INTERNAL_SUBNET="192.168.4.0/24" 
export CHAMELEON_INTERNAL_SUBNET_ESC="192.168.4.0\/24"  
export CHAMELEON_INTERNAL_NET_INTERNET_GATEWAY_IP="192.168.4.254"
export CHAMELEON_INTERNAL_NET_DHCP_ALLOCATION_START="192.168.4.100"
export CHAMELEON_INTERNAL_NET_DHCP_ALLOCATION_END="192.168.4.200"

# AWS 
export AWS_ASN="65001"
export AWS_ROUTER_EXTERNAL_IP="192.168.3.2"
export AWS_ROUTER_EXTERNAL_IP_CIDR=${AWS_ROUTER_EXTERNAL_IP}/24
export AWS_BGP_PASSWORD="BgpAuthPass"

# Chameleon 
export CHAMELEON_ASN="65002"
export CHAMELEON_ROUTER_IP_FACING_AWS="192.168.3.1"
export CHAMELEON_ROUTER_IP_FACING_INTERNAL="192.168.4.1"
export CHAMELEON_ROUTER_IP_FACING_AWS_CIDR=${CHAMELEON_ROUTER_IP_FACING_AWS}/24

export BGP_PASSWORD="BgpAuthPass"


# AWS Configuration
# To get your keys see: https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html
# The following assumes the .csv aquired from AWS is placed in your notebook's ~/work folder but hardcoding 
# the values here will also work.
AWS_ACCESS_KEY_ID=`tail -n +2  ~/work/new_user_credentials.csv | cut -d, -f1`
AWS_SECRRET_ACCESS_KEY=`tail -n +2  ~/work/new_user_credentials.csv | cut -d, -f2`

DEFAULT_REGION_NAME='us-east-2'
DEFAULT_OUTPUT_FORMAT='json'







### Chameleon Configuration

First, we show how the create the Chameleon resources.

#### Create Chameleon networks

On Chameleon there are two separate networks. The external network connects to Internet2 while the internal network connects local Chameleon nodes. Later we will creata a BGP router that will route network traffic between these networks.

##### Create the external network

The external network must be able to stitch to an external exogeni stitchport. This is achomplished by reserving an externally stitchable isolated network and setting the provider network to exogeni. We will be managing our own router for this network and do not need Chameleon to create one for us. 

In [3]:
#Set the names of the lease and network
EXTERNAL_NET_LEASE_NAME="${PREFACE}-AWS-Network-External-Lease"
EXTERNAL_NET_NAME="${PREFACE}-AWS-Network-External-Network"

#Set the provider network 
PROVIDER="exogeni"

blazar lease-create \
   --reservation resource_type=network,network_name=${EXTERNAL_NET_NAME},resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
   ${EXTERNAL_NET_LEASE_NAME}
   

Matches: ('resource_type=network,network_name=pruth-AWS-Network-External-Network', 'resource_properties', '["==","$physical_network","exogeni"]')
Matches: ('resource_type=network', 'network_name', 'pruth-AWS-Network-External-Network')
Matches: (None, 'resource_type', 'network')
Created a new lease:
+--------------+------------------------------------------------------------------+
| Field        | Value                                                            |
+--------------+------------------------------------------------------------------+
| created_at   | 2020-09-25 16:18:42                                              |
| degraded     | False                                                            |
| end_date     | 2020-09-26T16:18:00.000000                                       |
| events       | {                                                                |
|              |     "status": "UNDONE",                                          |
|              |     "lease_

In [4]:
EXTERNAL_SUBNET_NAME="${PREFACE}-AWS-Network-External-Subnet"
 
openstack subnet create --subnet-range ${EXTERNAL_SUBNET} \
                   --no-dhcp \
                   --gateway ${EXTERNAL_NET_INTERNET_GATEWAY_IP} \
                   --network ${EXTERNAL_NET_NAME} ${EXTERNAL_SUBNET_NAME}

+----------------------+--------------------------------------+
| Field                | Value                                |
+----------------------+--------------------------------------+
| allocation_pools     | 192.168.3.1-192.168.3.253            |
| cidr                 | 192.168.3.0/24                       |
| created_at           | 2020-09-25T16:19:06Z                 |
| description          |                                      |
| dns_nameservers      |                                      |
| dns_publish_fixed_ip | None                                 |
| enable_dhcp          | False                                |
| gateway_ip           | 192.168.3.254                        |
| host_routes          |                                      |
| id                   | 325b816e-71fa-4c5a-8cd5-73e8acc492d3 |
| ip_version           | 4                                    |
| ipv6_address_mode    | None                                 |
| ipv6_ra_mode         | None           

##### Create the internal network

The internal network connects to local Chameleon nodes.  This is achomplished by reserving an isolated network and setting the provider network to physnet1. The subnet and router for the internal network will be used by the nodes to access the public Internet.

The internal network can, optionally, be an OpenFlow network or even a stichable network. The example below uses a standared isolated VLAN.

In [5]:
#Set the names of the lease and network
INTERNAL_NET_LEASE_NAME="${USER}-AWS-Network-Internal-Lease"
INTERNAL_NET_NAME="${USER}-AWS-Network-Internal-Name"

#Set the provider network 
PROVIDER="physnet1"
    
blazar lease-create \
   --reservation resource_type=network,network_name=${INTERNAL_NET_NAME},resource_properties="[\"==\",\"\$physical_network\",\"$PROVIDER\"]" \
   ${INTERNAL_NET_LEASE_NAME}

Matches: ('resource_type=network,network_name=pruth-AWS-Network-Internal-Name', 'resource_properties', '["==","$physical_network","physnet1"]')
Matches: ('resource_type=network', 'network_name', 'pruth-AWS-Network-Internal-Name')
Matches: (None, 'resource_type', 'network')
Created a new lease:
+--------------+------------------------------------------------------------------+
| Field        | Value                                                            |
+--------------+------------------------------------------------------------------+
| created_at   | 2020-09-25 16:19:11                                              |
| degraded     | False                                                            |
| end_date     | 2020-09-26T16:19:00.000000                                       |
| events       | {                                                                |
|              |     "status": "UNDONE",                                          |
|              |     "lease_id": 

In [6]:
INTERNAL_NET_SUBNET_NAME="${PREFACE}-AWS-Network-Internal-Subnet"
INTERNAL_NET_ROUTER_NAME="${PREFACE}-AWS-Network-Internal-Router"
                   
openstack subnet create --subnet-range ${CHAMELEON_INTERNAL_SUBNET} \
                   --dhcp \
                   --allocation-pool start=${CHAMELEON_INTERNAL_NET_DHCP_ALLOCATION_START},end=${CHAMELEON_INTERNAL_NET_DHCP_ALLOCATION_END} \
                   --gateway ${CHAMELEON_INTERNAL_NET_INTERNET_GATEWAY_IP} \
                   --network ${INTERNAL_NET_NAME} \
                   ${INTERNAL_NET_SUBNET_NAME}
                   
# Set the name of the public network for Internet access
PUBLIC_NET="public"

openstack router create ${INTERNAL_NET_ROUTER_NAME}
openstack router add subnet ${INTERNAL_NET_ROUTER_NAME} ${INTERNAL_NET_SUBNET_NAME} 
openstack router set --external-gateway ${PUBLIC_NET} ${INTERNAL_NET_ROUTER_NAME}

# Set an external subnet that will be routed to the local BGP gateway instead of the default gateway.
# This subnet must include the subnet used on AWS.
openstack subnet set --host-route destination=${AWS_INTERNAL_SUBNET},gateway=${CHAMELEON_ROUTER_IP_FACING_INTERNAL} ${INTERNAL_NET_SUBNET_NAME}

+----------------------+--------------------------------------+
| Field                | Value                                |
+----------------------+--------------------------------------+
| allocation_pools     | 192.168.4.100-192.168.4.200          |
| cidr                 | 192.168.4.0/24                       |
| created_at           | 2020-09-25T16:19:49Z                 |
| description          |                                      |
| dns_nameservers      |                                      |
| dns_publish_fixed_ip | None                                 |
| enable_dhcp          | True                                 |
| gateway_ip           | 192.168.4.254                        |
| host_routes          |                                      |
| id                   | a6c69f2c-32b9-44b1-b066-6a4c82058370 |
| ip_version           | 4                                    |
| ipv6_address_mode    | None                                 |
| ipv6_ra_mode         | None           

#### Create a lease for floating IP addresses.

In [7]:
FLOATING_IP_LEASE_NAME="${USER}-AWS-BGP-FloatingIP"

AMOUNT="1"

PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

blazar lease-create \
  --reservation "resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=${AMOUNT}" \
  "$FLOATING_IP_LEASE_NAME"


Matches: ('resource_type=virtual:floatingip,network_id=44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '44b38c44-2a42-4b6d-b129-6c8f1b2a1375')
Matches: (None, 'resource_type', 'virtual:floatingip')
Created a new lease:
+--------------+------------------------------------------------------------------+
| Field        | Value                                                            |
+--------------+------------------------------------------------------------------+
| created_at   | 2020-09-25 16:20:30                                              |
| degraded     | False                                                            |
| end_date     | 2020-09-26T16:20:00.000000                                       |
| events       | {                                                                |
|              |     "status": "UNDONE",                                          |
|              |     "lease_id": "d11a9733-

#### Create the BGP router 
In this setup, 2 instances are created. 
	BGP speaker (sw-bgp) is an instance that is built on a dual-nic Haswell node.
Regular node (sw-instance-1) in the subnet can be built on a Skylake or Haswell node.

Create a lease with 2-3 dual-nic Haswell nodes. 


In [8]:
# Create Lease for the BGP router.
# Requires 1 Haswell Node with dual-NICs enabled
#
BGP_ROUTER_LEASE_NAME="${USER}-AWS-BGP-Router-Lease" 

NODE_TYPE="compute_haswell"
MIN=1
MAX=1

blazar lease-create \
      --physical-reservation min=${MIN},max=${MAX},resource_properties="[\"and\",[\"==\",\"\$network_adapters.1.enabled\",\"True\"],[\"==\",\"\$node_type\",\"compute_haswell\"]]" \
      ${BGP_ROUTER_LEASE_NAME}
      

Matches: ('min=1,max=1', 'resource_properties', '["and",["==","$network_adapters.1.enabled","True"],["==","$node_type","compute_haswell"]]')
Matches: ('min=1', 'max', '1')
Matches: (None, 'min', '1')
Created a new lease:
+--------------+---------------------------------------------------------------------------------------------------------------------------------------+
| Field        | Value                                                                                                                                 |
+--------------+---------------------------------------------------------------------------------------------------------------------------------------+
| created_at   | 2020-09-25 16:20:42                                                                                                                   |
| degraded     | False                                                                                                                                 |
| end_date    

In [39]:
# Get the reservation ID
BGP_ROUTER_RESERVATION_ID=$(blazar lease-show  -f json ${BGP_ROUTER_LEASE_NAME} | jq -r .reservations | jq -r .id)
BGP_ROUTER_LEASE_ID=$(blazar lease-show  -f json ${BGP_ROUTER_LEASE_NAME} | jq -r .id)

# Get network UUIDs
NET_UUID_INTERNAL=$( openstack network show -f value -c id ${INTERNAL_NET_NAME} )
NET_UUID_EXTERNAL=$( openstack network show -f value -c id ${EXTERNAL_NET_NAME} )

BGP_ROUTER_NAME="${USER}-AWS-BGP-Router" 

FLAVOR="baremetal"
IMAGE="CC-CentOS7"

openstack server create \
  --image ${IMAGE} \
  --flavor ${FLAVOR} \
  --key-name ${KEY_NAME} \
  --nic net-id=${NET_UUID_INTERNAL},v4-fixed-ip=${CHAMELEON_ROUTER_IP_FACING_INTERNAL} \
  --nic net-id=${NET_UUID_EXTERNAL},v4-fixed-ip=${CHAMELEON_ROUTER_IP_FACING_AWS} \
  --hint reservation=${BGP_ROUTER_RESERVATION_ID} \
  --hint query='["=","$hypervisor_hostname","$PHYSICAL"]' \
  ${BGP_ROUTER_NAME}

+-------------------------------------+------------------------------------------------------------------+
| Field                               | Value                                                            |
+-------------------------------------+------------------------------------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                                           |
| OS-EXT-AZ:availability_zone         |                                                                  |
| OS-EXT-SRV-ATTR:host                | None                                                             |
| OS-EXT-SRV-ATTR:hypervisor_hostname | None                                                             |
| OS-EXT-SRV-ATTR:instance_name       |                                                                  |
| OS-EXT-STS:power_state              | NOSTATE                                                          |
| OS-EXT-STS:task_state              

#### Associate floating IPs with instances.

You must wait for the node to start spawning before you can associate a floating IP. 


In [40]:
BGP_ROUTER_FLOATING_IP=$(lease_list_floating_ips "$FLOATING_IP_LEASE_NAME" | sed -n 1p)

openstack server add floating ip "$BGP_ROUTER_NAME" "$BGP_ROUTER_FLOATING_IP" \
  && echo "Attached floating ip $BGP_ROUTER_FLOATING_IP!"

Attached floating ip 192.5.87.172!


#### Configure BGP Router

The following script will configure the BGP router node and install a docker container that will communicate with the Internet2 CloudConnect BGP router.


In [41]:
login_command="ssh -o \"StrictHostKeyChecking no\" -i $PRIVATE_KEY_LOCATION cc@$BGP_ROUTER_FLOATING_IP"

IF_INTERNAL_NET='eth0'
IF_EXTERNAL_NET='eth1'

CHAMELEON_ROUTER_IP_FACING_AWS_CIDR='${CHAMELEON_ROUTER_IP_FACING_AWS}\/24'
CHAMELEON_ROUTER_IP_FACING_INTERNAL_CIDR='${CHAMELEON_ROUTER_IP_FACING_INTERNAL}\/24'

eval "$login_command bash << EOF

echo Configure external iface
sudo ip addr add ${CHAMELEON_ROUTER_IP_FACING_AWS}/24 dev ${IF_EXTERNAL_NET}

sudo ip link set ${IF_INTERNAL_NET} up
sudo ip link set ${IF_INTERNAL_NET} mtu 9000
sudo ip link set ${IF_INTERNAL_NET} txqueuelen 10000

sudo ip link set ${IF_EXTERNAL_NET} up
sudo ip link set ${IF_EXTERNAL_NET} mtu 9000
sudo ip link set ${IF_EXTERNAL_NET} txqueuelen 10000

sudo iptables -I FORWARD -i ${IF_EXTERNAL_NET} -o ${IF_INTERNAL_NET} -m state --state NEW,ESTABLISHED,RELATED -j ACCEPT
sudo iptables -I FORWARD -i ${IF_INTERNAL_NET} -o ${IF_EXTERNAL_NET} -m state --state NEW,ESTABLISHED,RELATED -j ACCEPT 

#Remove the external route because this is the router
sudo ip route del ${AWS_INTERNAL_SUBNET}

sudo sh -c 'echo net.core.rmem_max = 67108864 >> /etc/sysctl.conf'
sudo sh -c 'echo net.core.wmem_max = 67108864  >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_rmem = 4096 87380 33554432 >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_wmem = 4096 65536 33554432 >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_congestion_control=htcp >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_mtu_probing=1 >> /etc/sysctl.conf'
sudo sh -c 'echo net.core.default_qdisc = fq >> /etc/sysctl.conf'

sudo sysctl -p 

#
# Create Quagga and CorsaCRA instance

sudo yum update -y
sudo yum install -y yum-utils device-mapper-persistent-data lvm2

sudo yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo
sudo yum install -y docker-ce

sudo systemctl enable docker
sudo systemctl start docker
sudo systemctl status docker

yum install -y git
mkdir ~/corsa_cra
cd ~/corsa_cra/ 
git init 
git config core.sparsecheckout true 
echo corsa_cra/corsa_cra/* >> .git/info/sparse-checkout 
git remote add -f origin https://mcevik0@github.com/RENCI-NRIG/exogeni-recipes.git 
git pull origin master 


cd ~/corsa_cra/corsa_cra/corsa_cra/docker/

# Modify corsa_cra/quagga/bgpd.conf

sed -r -i \"s/<LOCAL_ASN>/${CHAMELEON_ASN}/g\" corsa_cra/quagga/bgpd.conf
sed -r -i \"s/<LOCAL_ROUTER_IP>/${CHAMELEON_ROUTER_IP_FACING_AWS}/g\" corsa_cra/quagga/bgpd.conf
sed -r -i \"s/<LOCAL_SUBNET>/${CHAMELEON_INTERNAL_SUBNET_ESC}/g\" corsa_cra/quagga/bgpd.conf
sed -r -i \"s/<REMOTE_ROUTER_IP>/${AWS_ROUTER_EXTERNAL_IP}/g\" corsa_cra/quagga/bgpd.conf
sed -r -i \"s/<REMOTE_ASN>/${AWS_ASN}/g\" corsa_cra/quagga/bgpd.conf
sed -r -i \"s/<REMOTE_DESC>/AWS/g\" corsa_cra/quagga/bgpd.conf
sed -r -i \"s/<BGP_PASSWORD>/${BGP_PASSWORD}/g\" corsa_cra/quagga/bgpd.conf

# Modify  corsa_cra/quagga/zebra.conf

sed -r -i \"s/<INTERFACE_FACING_AWS>/${IF_EXTERNAL_NET}/g\" corsa_cra/quagga/zebra.conf
sed -r -i \"s/<INTERFACE_FACING_LOCAL>/${IF_INTERNAL_NET}/g\" corsa_cra/quagga/zebra.conf
sed -r -i \"s/<IP_ADDRESS_FACING_AWS>/${CHAMELEON_ROUTER_IP_FACING_AWS_CIDR}/g\" corsa_cra/quagga/zebra.conf
sed -r -i \"s/<IP_ADDRESS_FACING_LOCAL>/${CHAMELEON_ROUTER_IP_FACING_INTERNAL_CIDR}/g\" corsa_cra/quagga/zebra.conf

cd ~/corsa_cra/corsa_cra/corsa_cra/docker/ 
sudo docker build -t cra_2 . 
sudo docker run --rm -dit --privileged --network host -p 6653:6653 --name=cra_2 cra_2 
sudo docker image ls

EOF"

Configure external iface
net.core.rmem_max = 67108864
net.core.wmem_max = 67108864
net.ipv4.tcp_rmem = 4096 87380 33554432
net.ipv4.tcp_wmem = 4096 65536 33554432
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq
Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: bay.uchicago.edu
 * epel: ord.mirror.rackspace.com
 * extras: mirror.mobap.edu
 * updates: mirror.atlanticmetro.net
Resolving Dependencies
--> Running transaction check
---> Package python2-pip.noarch 0:8.1.2-12.el7 will be updated
---> Package python2-pip.noarch 0:8.1.2-14.el7 will be an update
--> Finished Dependency Resolution

Dependencies Resolved

 Package              Arch            Version               Repository     Size
Updating:
 python2-pip          noarch          8.1.2-14.el7          epel          1.7 M

Transaction Summary
Upgrade  1 Package

Total download size: 1.7 M
Delta RPMs disabled because /usr/bin/applydeltarpm not installed.

### Add local compute nodes

At this point the internal network is 


In [10]:
NODES_FLOATING_IP_LEASE_NAME="${USER}-Node-FloatingIP"

AMOUNT="1"

PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

blazar lease-create \
  --reservation "resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=${AMOUNT}" \
  "$NODES_FLOATING_IP_LEASE_NAME"


Matches: ('resource_type=virtual:floatingip,network_id=44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '44b38c44-2a42-4b6d-b129-6c8f1b2a1375')
Matches: (None, 'resource_type', 'virtual:floatingip')
Created a new lease:
+--------------+------------------------------------------------------------------+
| Field        | Value                                                            |
+--------------+------------------------------------------------------------------+
| created_at   | 2020-09-25 16:22:15                                              |
| degraded     | False                                                            |
| end_date     | 2020-09-26T16:22:00.000000                                       |
| events       | {                                                                |
|              |     "status": "UNDONE",                                          |
|              |     "lease_id": "a33ca7fd-

#### Create servers on the new network

You can now create Chameleon nodes on the internal network that will be able to communicate over the CloudConnect link that we will create later in this tutorial. Any nodes can be created on the internal network. The example creates a single x86 haswell node.

In [11]:
# Create Lease for the BGP router.
# Requires 1 Haswell Node with dual-NICs enabled
#
NODES_LEASE_NAME="${USER}-AWS-Nodes-Lease" 

NODE_TYPE="compute_haswell"
MIN=1
MAX=1

blazar lease-create \
      --physical-reservation min=${MIN},max=${MAX},resource_properties="[\"==\",\"\$node_type\",\"compute_haswell\"]" \
      ${NODES_LEASE_NAME}

Matches: ('min=1,max=1', 'resource_properties', '["==","$node_type","compute_haswell"]')
Matches: ('min=1', 'max', '1')
Matches: (None, 'min', '1')
Created a new lease:
+--------------+---------------------------------------------------------------------------+
| Field        | Value                                                                     |
+--------------+---------------------------------------------------------------------------+
| created_at   | 2020-09-25 16:22:27                                                       |
| degraded     | False                                                                     |
| end_date     | 2020-09-26T16:22:00.000000                                                |
| events       | {                                                                         |
|              |     "status": "UNDONE",                                                   |
|              |     "lease_id": "c2eae580-1dff-47ca-8883-baedf8e17e5d",               

In [12]:
# Get the reservation ID
NODES_RESERVATION_ID=$(blazar lease-show  -f json ${NODES_LEASE_NAME} | jq -r .reservations | jq -r .id)
NODES_LEASE_ID=$(blazar lease-show  -f json ${NODES_LEASE_NAME} | jq -r .id)

# Get network UUIDs
NET_UUID_INTERNAL=$( openstack network show -f value -c id ${INTERNAL_NET_NAME} )

NODES_NAME="${USER}-Node" 

FLAVOR="baremetal"
IMAGE="CC-CentOS7"

openstack server create \
  --image ${IMAGE} \
  --flavor ${FLAVOR} \
  --key-name ${KEY_NAME} \
  --nic net-id=${NET_UUID_INTERNAL} \
  --hint reservation=${NODES_RESERVATION_ID} \
  --hint query='["=","$hypervisor_hostname","$PHYSICAL"]' \
  ${NODES_NAME}

+-------------------------------------+------------------------------------------------------------------+
| Field                               | Value                                                            |
+-------------------------------------+------------------------------------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                                           |
| OS-EXT-AZ:availability_zone         |                                                                  |
| OS-EXT-SRV-ATTR:host                | None                                                             |
| OS-EXT-SRV-ATTR:hypervisor_hostname | None                                                             |
| OS-EXT-SRV-ATTR:instance_name       |                                                                  |
| OS-EXT-STS:power_state              | NOSTATE                                                          |
| OS-EXT-STS:task_state              

In [13]:
NODE_FLOATING_IP=$(lease_list_floating_ips "$NODES_FLOATING_IP_LEASE_NAME" | sed -n 1p)

openstack server add floating ip "$NODES_NAME" "$NODE_FLOATING_IP" \
  && echo "Attached floating ip $NODE_FLOATING_IP!"

Attached floating ip 192.5.87.211!


It is probably worth configuring the node with jumbo frames for better performance. The following shows the network tuning we used to get the most bandwidth. These tunning parameters are recommended by [ESnet](https://fasterdata.es.net/host-tuning/linux/).

In [18]:
login_command="ssh -o \"StrictHostKeyChecking no\" -i $PRIVATE_KEY_LOCATION cc@$NODE_FLOATING_IP"

eval "$login_command bash << EOF

sudo ip link set eth0 mtu 9000
sudo ip link set eth0 txqueuelen 10000

# allow testing with buffers up to 64MB 

sudo sh -c 'echo net.core.rmem_max = 67108864 >> /etc/sysctl.conf'
sudo sh -c 'echo net.core.wmem_max = 67108864  >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_rmem = 4096 87380 33554432 >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_wmem = 4096 65536 33554432 >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_congestion_control=htcp >> /etc/sysctl.conf'
sudo sh -c 'echo net.ipv4.tcp_mtu_probing=1 >> /etc/sysctl.conf'
sudo sh -c 'echo net.core.default_qdisc = fq >> /etc/sysctl.conf'

sudo sysctl -p 

EOF"

net.core.rmem_max = 67108864
net.core.wmem_max = 67108864
net.ipv4.tcp_rmem = 4096 87380 33554432
net.ipv4.tcp_wmem = 4096 65536 33554432
net.ipv4.tcp_congestion_control = htcp
net.ipv4.tcp_mtu_probing = 1
net.core.default_qdisc = fq


### Internet2 CloudConnect 

The Internet2 CloudConnect connection must be created by an authorized administrator. We would be happy to create this connection for you. Please create a Chameleon support ticket and cut/paste the output of the following cell. 

You will need to add your AWS account number and the desired bandwith.

If you are an authorized CloudConnect user and would like to be able to create your own links, please create a support ticket and we can disscuss support your request.


In [21]:
DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${EXTERNAL_NET_NAME}`

echo Bandwidth: See: https://aws.amazon.com/directconnect/pricing/
echo Connection Name: $DIRECT_CONNECT_NAME

echo AWS Account ID: Your AWS Account Number
echo AWS VLAN: $AWS_VLAN
echo AWS ASN: $AWS_ASN
echo AWS Router Extenal IP: $AWS_ROUTER_EXTERNAL_IP
echo Internet2 IP Facing AWS: $INTERNET2_ROUTER_IP_FACING_AWS
echo AWS Subnet: $AWS_SUBNET
echo AWS BGP Key: $AWS_BGP_PASSWORD

echo Chameleon network VLAN: $DIRECTSTITCH_VLAN
echo Chameleon Router External IP: $CHAMELEON_ROUTER_IP_FACING_INTERNET2
echo Internet2 IP Facing AWS: $INTERNET2_ROUTER_IP_FACING_CHAMELEON
echo Chameleon Router External SUBNET: $EXTERNAL_SUBNET
echo Chameleon ASN: $CHAMELEON_ASN
echo Chameleon Router AuthPassword: $CHAMELEON_BGP_PASSWORD
echo Chameleon Region: $OS_REGION_NAME



Unauthorized (HTTP 401)
Bandwidth: See: https://aws.amazon.com/directconnect/pricing/
Connection Name: Chameleon_Direct_Connect
AWS Account ID: Your AWS Account Number
AWS VLAN: 10
AWS ASN: 65001
AWS Router Extenal IP: 192.168.3.2
Internet2 IP Facing AWS:
AWS Subnet:
AWS BGP Key: BgpAuthPass
Chameleon network VLAN:
Chameleon Router External IP:
Internet2 IP Facing AWS:
Chameleon Router External SUBNET: 192.168.3.0/24
Chameleon ASN: 65002
Chameleon Router AuthPassword: BgpAuthPass
Chameleon Region: CHI@UC


### Set up AWS Direct Connect Connection

In [24]:
#sudo yum -y install awscli

AWS_ACCESS_KEY_ID=`tail -n +2  ~/work/new_user_credentials.csv | cut -d, -f1`
AWS_SECRRET_ACCESS_KEY=`tail -n +2  ~/work/new_user_credentials.csv | cut -d, -f2`

DEFAULT_REGION_NAME='us-east-2'
DEFAULT_OUTPUT_FORMAT='json'


# Note: can be done internativly with 'aws configure'
aws configure set aws_access_key_id $AWS_ACCESS_KEY_ID
aws configure set aws_secret_access_key $AWS_SECRRET_ACCESS_KEY
aws configure set default.region $DEFAULT_REGION_NAME
aws configure set default.output $DEFAULT_OUTPUT_FORMAT


In [25]:

#Needs try/catch b/c some vpcs don't have Tags/Names
VPC_ID=`aws --output json ec2 describe-vpcs | jq -r '.Vpcs[] | try if .Tags[].Key == "Name" and .Tags[].Value == "'${AWS_VPC_NAME}'" then .VpcId else "" end catch ""'`

echo VPC_ID $VPC_ID

VPC_ID vpc-095801bc19a70cdbc


In [26]:
#Create VPG
VPC_GATEWAY_ID=`aws --output json ec2 create-vpn-gateway --type ipsec.1 --amazon-side-asn 65001 | jq -r '.VpnGateway.VpnGatewayId'`
echo VPC_GATEWAY_ID $VPC_GATEWAY_ID
                                        
                                        

VPC_GATEWAY_ID vgw-0939d968c1d9b3665


In [27]:
#Attach VPG to VPC
aws --output json ec2 attach-vpn-gateway --vpc-id $VPC_ID --vpn-gateway-id $VPC_GATEWAY_ID


{
    "VpcAttachment": {
        "State": "attaching",
        "VpcId": "vpc-095801bc19a70cdbc"
    }
}


In [28]:
ROUTE_TABLE_ID=`aws --output json ec2 describe-route-tables | jq -r '.RouteTables[] |  try if .VpcId == "'${VPC_ID}'" then .RouteTableId else "" end catch ""'`
echo ROUTE_TABLE_ID $ROUTE_TABLE_ID

ROUTE_TABLE_ID rtb-0434d4438f34a2460


In [29]:

aws --output json ec2 enable-vgw-route-propagation --gateway-id $VPC_GATEWAY_ID \
                                                   --route-table-id $ROUTE_TABLE_ID

In [30]:
#Get directconnect conneciton id
DIRECT_CONNECT_ID=`aws --output json directconnect describe-connections | jq -r '.connections[] |  try if .connectionName == "'${DIRECT_CONNECT_NAME}'" then .connectionId else "" end catch ""'`
echo DIRECTCONNECT_ID $DIRECT_CONNECT_ID


DIRECTCONNECT_ID dxcon-fh173erj


In [31]:
aws --output json directconnect confirm-connection --connection-id $DIRECT_CONNECT_ID

{
    "connectionState": "pending"
}


In [32]:
DIRECT_CONNECT_STATE=""

while [ "$DIRECT_CONNECT_STATE" != "available" ]
do 
    sleep 10
    #Needs try/catch b/c some instnaces don't have Tags/Names
    DIRECT_CONNECT_STATE=`aws --output json directconnect describe-connections | jq -r '.connections[] |  try if .connectionName == "'${DIRECT_CONNECT_NAME}'" then .connectionState else "" end catch ""'`
    echo DIRECTCONNECT_STATE $DIRECT_CONNECT_STATE
done

DIRECTCONNECT_STATE pending
DIRECTCONNECT_STATE available


In [33]:
#Create DirectConnectGateway
DIRECT_CONNECT_GATEWAY_NAME="dcgw-"${DIRECT_CONNECT_NAME} 

aws --output json directconnect create-direct-connect-gateway --direct-connect-gateway-name $DIRECT_CONNECT_GATEWAY_NAME --amazon-side-asn $AWS_ASN

{
    "directConnectGateway": {
        "directConnectGatewayId": "85cca4bb-770a-4747-8f22-9227f85a946c",
        "directConnectGatewayName": "dcgw-Chameleon_Direct_Connect",
        "amazonSideAsn": 65001,
        "ownerAccount": "370592084203",
        "directConnectGatewayState": "available"
    }
}


In [34]:
#Needs try/catch b/c some instnaces don't have Tags/Names
DIRECT_CONNECT_GW_ID=`aws --output json directconnect describe-direct-connect-gateways | jq -r '.directConnectGateways[] |  try if .directConnectGatewayName == "'${DIRECT_CONNECT_GATEWAY_NAME}'" then .directConnectGatewayId else "" end catch ""'`
echo DIRECTCONNECT_GW_ID $DIRECT_CONNECT_GW_ID

DIRECTCONNECT_GW_ID 85cca4bb-770a-4747-8f22-9227f85a946c


In [36]:
#Create VirtualPrivateGateway
VIRTUAL_INTERFACE_NAME="vif-"${DIRECT_CONNECT_NAME} 

MTU=9001

aws --output json directconnect create-private-virtual-interface \
                                          --connection-id $DIRECT_CONNECT_ID \
                                          --new-private-virtual-interface virtualInterfaceName=${VIRTUAL_INTERFACE_NAME},vlan=${AWS_VLAN},asn=${CHAMELEON_ASN},authKey=${BGP_PASSWORD},mtu=${MTU},amazonAddress=${AWS_ROUTER_EXTERNAL_IP_CIDR},customerAddress=${CHAMELEON_ROUTER_IP_FACING_AWS_CIDR},directConnectGatewayId=${DIRECT_CONNECT_GW_ID}
                                          
                                          

{
    "ownerAccount": "370592084203",
    "virtualInterfaceId": "dxvif-fgzvrw7f",
    "location": "EQC50",
    "connectionId": "dxcon-fh173erj",
    "virtualInterfaceType": "private",
    "virtualInterfaceName": "vif-Chameleon_Direct_Connect",
    "vlan": 10,
    "asn": 65002,
    "amazonSideAsn": 65001,
    "authKey": "BgpAuthPass",
    "amazonAddress": "192.168.3.2/24",
    "customerAddress": "192.168.3.1/24",
    "addressFamily": "ipv4",
    "virtualInterfaceState": "pending",
    "customerRouterConfig": "<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n<logical_connection id=\"dxvif-fgzvrw7f\">\n  <vlan>10</vlan>\n  <customer_address>192.168.3.1/24</customer_address>\n  <amazon_address>192.168.3.2/24</amazon_address>\n  <bgp_asn>65002</bgp_asn>\n  <bgp_auth_key>BgpAuthPass</bgp_auth_key>\n  <amazon_bgp_asn>65001</amazon_bgp_asn>\n  <connection_type>private</connection_type>\n</logical_connection>\n",
    "mtu": 9001,
    "jumboFrameCapable": true,
    "virtualGatewayId": "",
    "direct

In [37]:
#Associate VPGW with DirectConnect GW

aws --output json  directconnect create-direct-connect-gateway-association --direct-connect-gateway-id $DIRECT_CONNECT_GW_ID \
                                                                           --virtual-gateway-id $VPC_GATEWAY_ID

{
    "directConnectGatewayAssociation": {
        "directConnectGatewayId": "85cca4bb-770a-4747-8f22-9227f85a946c",
        "directConnectGatewayOwnerAccount": "370592084203",
        "associationState": "associating",
        "associatedGateway": {
            "id": "vgw-0939d968c1d9b3665",
            "type": "virtualPrivateGateway",
            "ownerAccount": "370592084203",
            "region": "us-east-2"
        },
        "associationId": "ec12df56-fbdb-4870-8703-e504a4d3feb9",
        "allowedPrefixesToDirectConnectGateway": [
            {
                "cidr": "192.168.1.0/24"
            }
        ],
        "virtualGatewayId": "vgw-0939d968c1d9b3665",
        "virtualGatewayRegion": "us-east-2",
        "virtualGatewayOwnerAccount": "370592084203"
    }
}
